In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory



In [2]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


#DATASET

to include glove file in your kernel, simple click 'input files' at the top of the notebook, and search 'glove' in the 'datasets' section.

In [3]:

EMBEDDING_FILE='glove.6B.100d.txt'
TRAIN_DATA_FILE=f'friends-personality.csv'
#TEST_DATA_FILE=f'{path}{comp}test.csv'

In [4]:
#set parameters
embed_size = 100 # how big is each word vector
max_features = 5414 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 300 # max number of words in a comment to use

In [5]:
#read datasets
train = pd.read_csv(TRAIN_DATA_FILE)
print(train)

     Unnamed: 0    scene_id  ... cOPN cNEU
0             0  01_e01_c01  ...    0    1
1             1  01_e01_c02  ...    1    1
2             2  01_e01_c04  ...    0    1
3             3  01_e01_c05  ...    1    0
4             4  01_e01_c08  ...    1    1
..          ...         ...  ...  ...  ...
706         706  04_e24_c18  ...    0    0
707         707  04_e24_c19  ...    1    0
708         708  04_e24_c19  ...    1    0
709         709  04_e24_c20  ...    0    1
710         710  04_e24_c24  ...    1    1

[711 rows x 9 columns]


#PRE PROCESSAMENTO 

In [6]:
list_sentences_train = train["text"].fillna("_na_").values
list_classes = ["cAGR", "cCON", "cEXT", "cOPN", "cNEU"]
y = train[list_classes].values

#list_sentences_test = test["text"].fillna("_na_").values

In [7]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
#list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
#X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

#WORD EMBBEDING

Read the glove word vectors (space delimited strings) into a dictionary from word->vector.

In [8]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

Use these vectors to create our embedding matrix, with random initialization for words that aren't in GloVe. We'll use the same mean and stdev of embeddings the GloVe has when generating the random init.

In [9]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


(0.004451992, 0.4081574)

In [10]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [11]:
print(embed_size, max_features)

100 5414


#TREINO E TESTE

In [12]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(100, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(100, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(5, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
print(y.shape)
print(y)

(711, 5)
[[1 1 0 0 1]
 [1 0 1 1 1]
 [0 0 0 0 1]
 ...
 [1 0 1 1 0]
 [0 1 1 0 1]
 [1 0 0 1 1]]


In [14]:
model.fit(X_t, y, batch_size=32, epochs=10, validation_split=0.1);

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 639 samples, validate on 72 samples
Epoch 1/10
639/639 [==============================] - 13s 21ms/step - loss: 0.6914 - accuracy: 0.5421 - val_loss: 0.6906 - val_accuracy: 0.5667
Epoch 2/10
639/639 [==============================] - 12s 19ms/step - loss: 0.6823 - accuracy: 0.5606 - val_loss: 0.6872 - val_accuracy: 0.5417
Epoch 3/10
639/639 [==============================] - 12s 19ms/step - loss: 0.6810 - accuracy: 0.5599 - val_loss: 0.6857 - val_accuracy: 0.5472
Epoch 4/10
639/639 [==============================] - 12s 19ms/step - loss: 0.6814 - accuracy: 0.5577 - val_loss: 0.6885 - val_accuracy: 0.5722
Epoch 5/10
639/639 [==============================] - 12s 19ms/step - loss: 0.6793 - accuracy: 0.5606 - val_loss: 0.6822 - val_accuracy: 0.5722
Epoch 6/10
639/639 [==============================] - 12s 19ms/step - loss: 0.6753 - accuracy: 0.5696 - val_loss: 0.6923 - val_accuracy: 0.5361
Epoch 7/10
639/639 [==============================] - 12s 19ms/step - loss: 0.6746 - accura